
# Telecom X – Parte 2: Predicción de Cancelación (Churn)

**Autor/a:** _Paula Almada_  

**Fecha:** 2025-08-09

Este notebook implementa un **pipeline de Machine Learning** completo para predecir **churn** en **Telecom X**, cumpliendo con todos los puntos indicados en la consigna:

- **Preparación de datos**: carga del archivo tratado (Parte 1), eliminación de columnas irrelevantes, encoding y estandarización cuando corresponde.  
- **Verificación de proporción de churn** y **balanceo** de clases (**SMOTE**) aplicado correctamente sólo sobre el conjunto de entrenamiento.  
- **Correlación y selección** de variables, más **análisis dirigido** (Contract × Churn, TotalCharges × Churn).  
- **Modelado** con al menos dos algoritmos (**Regresión Logística**, **Random Forest**) y uno adicional **KNN**.  
- **Evaluación** con Accuracy, Precision, Recall, F1, ROC-AUC y **matriz de confusión**, con comparación crítica y análisis de (under/over)fitting.  
- **Interpretación** de importancia de variables (coeficientes en Logística y feature_importances_ en Random Forest) y **conclusiones estratégicas**.


## 1) 📚 Librerías y configuración

In [ ]:

# Si ejecutas en Colab, descomenta para instalar imbalanced-learn
# !pip -q install imbalanced-learn

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    classification_report, ConfusionMatrixDisplay, roc_auc_score, roc_curve, accuracy_score,
    precision_score, recall_score, f1_score
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# imblearn para SMOTE y Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (7,5)



## 2) 📥 Carga del archivo tratado (Parte 1)

> **Usa el mismo CSV** que limpiaste y estandarizaste en la Parte 1. Debe contener **sólo columnas relevantes** y la columna objetivo **`Churn`** (0/1).


In [ ]:

# === Opción estándar: el archivo ya está en /content ===
# Sube 'datos_tratados.csv' a Colab (o monta Drive y ajusta la ruta).
df = pd.read_csv('/content/datos_tratados.csv')

print(f"Filas: {df.shape[0]} | Columnas: {df.shape[1]}")
display(df.head())
display(df.info())



## 3) 🧹 Preparación y verificación de datos

**3.1 Elimina columnas irrelevantes:** IDs y columnas anidadas que no aportan al modelo.  
**3.2 Define `X` e `y` y detecta tipos (numéricas/categóricas).  
**3.3 Proporción de churn y verificación de balanceo.**


In [ ]:

# 3.1 Columnas a eliminar (ajusta si tu dataset no las tiene)
cols_to_drop = ["customerID", "customer", "phone", "internet", "account"]
df = df.drop(columns=cols_to_drop, errors='ignore')

# 3.2 Definir target y features
target_col = "Churn"
assert target_col in df.columns, "No se encontró la columna 'Churn' en el dataset."

X = df.drop(columns=[target_col])
y = df[target_col].astype(int)

# Tipos
num_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object','category','bool']).columns.tolist()

print("Columnas numéricas:", len(num_cols))
print("Columnas categóricas:", len(cat_cols))


In [ ]:

# 3.3 Proporción de churn y chequeo de desbalance
prop = y.value_counts(normalize=True).rename({0:'No Churn', 1:'Churn'})
display(prop)
prop.plot(kind='bar', title='Proporción de clases (Churn)'); plt.show()



## 4) ✂️ Split y preprocesamiento

- **Split**: 70/30 estratificado.  
- **Preprocesamiento**:  
  - **One-Hot Encoding** para categóricas.  
  - **Estandarización** para numéricas **sólo** en modelos que lo requieren (Logística, KNN).  
  - **Árboles/Random Forest** no requieren escalado, pero lo incluimos sin perjuicio.  
- **SMOTE**: aplicado **sólo sobre el set de entrenamiento** dentro del pipeline para evitar fuga de información.


In [ ]:

# Split estratificado
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=42
)
print("Train:", X_train.shape, " Test:", X_test.shape)
print("Proporción churn (train/test):", round(y_train.mean(),3), "/", round(y_test.mean(),3))

# Preprocesadores
preprocess_scaled = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols)
    ],
    remainder='drop'
)

preprocess_noscale = ColumnTransformer(
    transformers=[
        ("num", "passthrough", num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols)
    ],
    remainder='drop'
)



## 5) 📊 Correlación y análisis dirigido

**5.1 Matriz de correlación** (numéricas + `Churn`).  
**5.2 Contract × Churn** (tasa de churn por tipo de contrato).  
**5.3 TotalCharges × Churn** (boxplot por clase).


In [ ]:

# 5.1 Matriz de correlación
if len(num_cols) > 0:
    corr = df[num_cols + [target_col]].corr()
    plt.figure()
    sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm")
    plt.title("Matriz de correlación (numéricas)")
    plt.show()

    corr_target = corr[target_col].drop(target_col).sort_values(ascending=False)
    display(corr_target.head(10))
else:
    print("No se detectaron columnas numéricas para correlación.")


In [ ]:

# 5.2 Contract × Churn
if 'Contract' in df.columns:
    rate_by_contract = df.groupby('Contract')[target_col].mean().sort_values(ascending=False)
    display(rate_by_contract)
    rate_by_contract.plot(kind='bar', title='Tasa de churn por tipo de contrato'); plt.show()
else:
    print("No se encontró la columna 'Contract' para el análisis dirigido.")


In [ ]:

# 5.3 TotalCharges × Churn (boxplot)
if 'TotalCharges' in df.columns:
    plt.figure()
    sns.boxplot(data=df, x=target_col, y='TotalCharges')
    plt.title("TotalCharges × Churn (boxplot)")
    plt.show()
else:
    print("No se encontró 'TotalCharges' para el análisis dirigido.")



## 6) 🤖 Modelado con balanceo de clases (SMOTE)

Entrenamos **tres** modelos:  
- **Regresión Logística** (requiere escalado).  
- **K-Nearest Neighbors (KNN)** (requiere escalado).  
- **Random Forest** (árboles, no requiere escalado).  

> SMOTE se aplica **dentro del pipeline**, **sólo al entrenamiento**.


In [ ]:

def evaluate_model(name, clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    if hasattr(clf, "predict_proba"):
        proba = clf.predict_proba(X_test)[:,1]
    else:
        # Para modelos sin predict_proba (no aplica aquí), usamos decision_function si existe
        proba = None

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds)
    rec = recall_score(y_test, preds)
    f1 = f1_score(y_test, preds)
    rocauc = roc_auc_score(y_test, proba) if proba is not None else np.nan

    print(f"=== {name} ===")
    print(f"Accuracy: {acc:.3f} | Precision: {prec:.3f} | Recall: {rec:.3f} | F1: {f1:.3f} | ROC-AUC: {rocauc:.3f}")
    print(classification_report(y_test, preds, digits=3))
    ConfusionMatrixDisplay.from_predictions(y_test, preds)
    plt.title(f"Matriz de confusión - {name}")
    plt.show()

    # ROC
    if proba is not None:
        fpr, tpr, _ = roc_curve(y_test, proba)
        plt.figure()
        plt.plot(fpr, tpr, label=f"{name} (AUC={rocauc:.3f})")
        plt.plot([0,1], [0,1], linestyle='--')
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title(f"Curva ROC - {name}")
        plt.legend()
        plt.show()

    return {"name": name, "accuracy": acc, "precision": prec, "recall": rec, "f1": f1, "rocauc": rocauc}

# Pipelines con SMOTE
log_reg = ImbPipeline(steps=[
    ("prep", preprocess_scaled),
    ("smote", SMOTE(random_state=42)),
    ("model", LogisticRegression(max_iter=1000))
])

knn = ImbPipeline(steps=[
    ("prep", preprocess_scaled),
    ("smote", SMOTE(random_state=42)),
    ("model", KNeighborsClassifier(n_neighbors=15))
])

rf = ImbPipeline(steps=[
    ("prep", preprocess_noscale),
    ("smote", SMOTE(random_state=42)),
    ("model", RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1))
])

results = []
results.append(evaluate_model("Logistic Regression (SMOTE)", log_reg, X_train, y_train, X_test, y_test))
results.append(evaluate_model("KNN (SMOTE)", knn, X_train, y_train, X_test, y_test))
results.append(evaluate_model("Random Forest (SMOTE)", rf, X_train, y_train, X_test, y_test))



## 7) 📈 Importancia de variables e interpretación

- **Random Forest:** `feature_importances_` (reducción de impureza).  
- **Regresión Logística:** coeficientes (signo y magnitud).  
> Para interpretar correctamente, usamos los **nombres de las columnas transformadas** tras el One-Hot Encoding.


In [ ]:

# Importancia Random Forest
ohe_rf = rf.named_steps['prep'].named_transformers_['cat']
cat_features_rf = ohe_rf.get_feature_names_out(input_features=rf.named_steps['prep'].transformers_[1][2]).tolist() if len(rf.named_steps['prep'].transformers_[1][2])>0 else []
num_features_rf = rf.named_steps['prep'].transformers_[0][2]
feature_names_rf = list(num_features_rf) + cat_features_rf

rf_importances = rf.named_steps['model'].feature_importances_
imp_df = pd.DataFrame({'Feature': feature_names_rf, 'Importance': rf_importances}).sort_values('Importance', ascending=False)
display(imp_df.head(20))

plt.figure(figsize=(8,6))
sns.barplot(data=imp_df.head(20), x='Importance', y='Feature')
plt.title("Top 20 variables más importantes - Random Forest")
plt.tight_layout()
plt.show()


In [ ]:

# Coeficientes Regresión Logística
ohe_lr = log_reg.named_steps['prep'].named_transformers_['cat']
cat_features_lr = ohe_lr.get_feature_names_out(input_features=log_reg.named_steps['prep'].transformers_[1][2]).tolist() if len(log_reg.named_steps['prep'].transformers_[1][2])>0 else []
num_features_lr = log_reg.named_steps['prep'].transformers_[0][2]
feature_names_lr = list(num_features_lr) + cat_features_lr

coefs = log_reg.named_steps['model'].coef_[0]
coef_df = pd.DataFrame({'Feature': feature_names_lr, 'Coef': coefs}).sort_values('Coef', ascending=False)
display(coef_df.head(15))
display(coef_df.tail(15))



## 8) 📊 Comparación de modelos y (under/over)fitting

Completa con tus resultados (copiando métricas de la sección de evaluación). Reflexiona:

- **Mejor desempeño** (¿cuál y por qué?).  
- **Overfitting** (alto rendimiento en train y pobre en test): posibles causas y mitigaciones.  
- **Underfitting** (bajo rendimiento general): acciones para mejorar.  

> Pistas:  
> - **Random Forest** suele rendir bien y ser robusto.  
> - **KNN** es sensible a la escala y al valor de `k`, y puede sufrir con muchas variables dummificadas.  
> - **Logística** es interpretable; revisa coeficientes y supuestos.



## 9) 📝 Conclusiones y recomendaciones

**Factores que más influyen en la cancelación**  
- Resume las variables top según **Random Forest** y los **coeficientes** de Logística (dirección + magnitud).

**Estrategias de retención**  
- Clientes con contrato **Month-to-month** y cargos mensuales altos.  
- Usuarios sin **TechSupport/OnlineSecurity**.  
- Ajusta campañas y ofertas según segmentos de riesgo.

**Siguientes pasos (opcional)**  
- Búsqueda de hiperparámetros (Grid/Random Search).  
- Probar **XGBoost/LightGBM**.  
- Calibración de probabilidades y ajuste del **umbral** de clasificación según costo/beneficio.


### Extra (opcional) – Búsqueda de hiperparámetros

In [ ]:

# from sklearn.model_selection import RandomizedSearchCV
# rf_search = ImbPipeline(steps=[('prep', preprocess_noscale), ('smote', SMOTE(random_state=42)), ('model', RandomForestClassifier(random_state=42, n_jobs=-1))])
# param_dist = {
#     'model__n_estimators': [200, 300, 500],
#     'model__max_depth': [None, 6, 10, 15],
#     'model__min_samples_split': [2, 5, 10],
#     'model__min_samples_leaf': [1, 2, 4]
# }
# rs = RandomizedSearchCV(rf_search, param_distributions=param_dist, n_iter=10, cv=3, scoring='roc_auc', n_jobs=-1, random_state=42)
# rs.fit(X_train, y_train)
# print("Mejores parámetros:", rs.best_params_)
# print("Mejor ROC-AUC (CV):", rs.best_score_)
